# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


- Nama: Rayhan Egar Sadtya Nugraha
- NIM: 225150201111014

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [4]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-26 19:40:54--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... 

connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.2’

iris.csv.2          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-26 19:40:55 (22.8 MB/s) - ‘iris.csv.2’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [6]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [7]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [8]:
print(data_latih.shape[0])
print(data_uji.shape[0])

120
30


In [9]:
data_uji.columns

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [10]:
label_uji = data_uji.pop('variety')

In [11]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
49,5.0,3.3,1.4,0.2
4,5.0,3.6,1.4,0.2
11,4.8,3.4,1.6,0.2
22,4.6,3.6,1.0,0.2
125,7.2,3.2,6.0,1.8


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data. 

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [12]:
from collections import Counter

def hitung_prior(list_kelas):
    n_data = len(list_kelas)
    prior = Counter(list_kelas)
    for key in prior.keys():
        prior[key] = prior[key]/n_data
    return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [13]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Versicolor': 0.35, 'Virginica': 0.3416666666666667, 'Setosa': 0.30833333333333335})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [14]:
def hitung_rata2_std_kelas(input_data):
    list_columns = input_data.columns[:-1]
    class_column_name = input_data.columns[-1]
    list_class = set(input_data[class_column_name])
    rata2 = {}
    std = {}
            
    for column in list_columns:
        for a_class in list_class:
            rata2[(a_class, column)] = input_data.loc[input_data[class_column_name]==a_class][column].mean()
            std[(a_class, column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
    
    return (rata2, std)

In [15]:
data_latih.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
140,6.7,3.1,5.6,2.4,Virginica
15,5.7,4.4,1.5,0.4,Setosa
36,5.5,3.5,1.3,0.2,Setosa
7,5.0,3.4,1.5,0.2,Setosa
96,5.7,2.9,4.2,1.3,Versicolor


Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [16]:
# test hitung_rata2_std_kelas()
test_rata2, test_std = hitung_rata2_std_kelas(data_latih)
print(test_rata2)
print(test_std)

{('Virginica', 'sepal.length'): 6.519512195121951, ('Setosa', 'sepal.length'): 5.0162162162162165, ('Versicolor', 'sepal.length'): 5.988095238095238, ('Virginica', 'sepal.width'): 2.9365853658536585, ('Setosa', 'sepal.width'): 3.4054054054054053, ('Versicolor', 'sepal.width'): 2.7928571428571427, ('Virginica', 'petal.length'): 5.524390243902438, ('Setosa', 'petal.length'): 1.462162162162162, ('Versicolor', 'petal.length'): 4.295238095238095, ('Virginica', 'petal.width'): 2.0121951219512195, ('Setosa', 'petal.width'): 0.2513513513513513, ('Versicolor', 'petal.width'): 1.3404761904761904}
{('Virginica', 'sepal.length'): 0.6546829431851429, ('Setosa', 'sepal.length'): 0.38406706121133, ('Versicolor', 'sepal.length'): 0.48449238255854854, ('Virginica', 'sepal.width'): 0.3389366441984221, ('Setosa', 'sepal.width'): 0.3535109191665372, ('Versicolor', 'sepal.width'): 0.3015349813002384, ('Virginica', 'petal.length'): 0.5855683084853884, ('Setosa', 'petal.length'): 0.16219218941214864, ('Versi

In [17]:
import math

def hitung_likelihood_gaussian(data, rata2, std):
    hasil = 1/(math.sqrt(2 * np.pi * np.power(std, 2))) * math.exp( (-np.power(data-rata2, 2))/ 2 * np.power(std, 2))
    return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-masing fitur dan kelas. 

In [18]:
def training_naive_bayes_gaussian(data_latih):

    class_column_name = data_latih.columns[-1]
    prior = hitung_prior(data)
    rata2, std = hitung_rata2_std_kelas(data_latih)
    list_class = set(data_latih[class_column_name])
    list_columns = data_latih.columns[:-1]
    
    model = {}

    model['prior'] = prior
    model['rata2'] = rata2
    model['std'] = std
    model['list_class'] = list_class
    model['list_columns'] = list_columns
    
    return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [19]:
model = training_naive_bayes_gaussian(data_latih)
print(model)

{'prior': Counter({'sepal.length': 0.006666666666666667, 'sepal.width': 0.006666666666666667, 'petal.length': 0.006666666666666667, 'petal.width': 0.006666666666666667, 'variety': 0.006666666666666667}), 'rata2': {('Virginica', 'sepal.length'): 6.519512195121951, ('Setosa', 'sepal.length'): 5.0162162162162165, ('Versicolor', 'sepal.length'): 5.988095238095238, ('Virginica', 'sepal.width'): 2.9365853658536585, ('Setosa', 'sepal.width'): 3.4054054054054053, ('Versicolor', 'sepal.width'): 2.7928571428571427, ('Virginica', 'petal.length'): 5.524390243902438, ('Setosa', 'petal.length'): 1.462162162162162, ('Versicolor', 'petal.length'): 4.295238095238095, ('Virginica', 'petal.width'): 2.0121951219512195, ('Setosa', 'petal.width'): 0.2513513513513513, ('Versicolor', 'petal.width'): 1.3404761904761904}, 'std': {('Virginica', 'sepal.length'): 0.6546829431851429, ('Setosa', 'sepal.length'): 0.38406706121133, ('Versicolor', 'sepal.length'): 0.48449238255854854, ('Virginica', 'sepal.width'): 0.33

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [20]:
def testing_naive_bayes_gaussian(model, data_uji):
    prior = model['prior']
    rata2 = model['rata2']
    std = model['std']
    list_class = model['list_class']
    list_columns = model['list_columns']
    posterior = dict.fromkeys(list_class, 1)

    for a_class in list_class:
        for column in list_columns:
            posterior[a_class] = posterior[a_class] * hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
        posterior[a_class] = posterior[a_class] * prior[a_class]
    kelas_uji = max(posterior, key=posterior.get)
    return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [21]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model, data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Virginica
Setosa


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [26]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
    prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total==label_uji))

Total prediksi benar:  9


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [22]:
! wget https://dataset-ppm.s3.amazonaws.com/car_sample.csv

--2024-03-26 19:40:56--  https://dataset-ppm.s3.amazonaws.com/car_sample.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.219.106.252, 52.219.179.236, 16.12.65.140, ...
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.219.106.252|:443... 

connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-03-26 19:40:58 ERROR 403: Forbidden.



In [27]:
import pandas as pd
data = pd.read_csv('car_sample.csv')
data.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [30]:
tugas_latih, tugas_uji = train_test_split(data, train_size=0.7, stratify=data['class'])

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [31]:
tugas_label_uji = tugas_uji.pop('class')

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [32]:
prior_tugas = hitung_prior(tugas_latih['class'])

Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [24]:
def hitung_likelihood_multinomial(data_latih):
    

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [25]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

NameError: name 'tugas_latih' is not defined

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [ ]:
def training_naive_bayes_multinomial(data_latih):
  #tulis kode Anda disini
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [ ]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [ ]:
def testing_naive_bayes_multinomial(model,data_uji):
  #tulis kode Anda disini
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [ ]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

good
good


Lakukan pengujian terhadap semua data uji.

In [ ]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  62


In [ ]:
len(tugas_label_uji)

78